# Alice at Intersection Example

In this notebook, we will describe the diagnostics process using an example inspired by the 2007 DARPA Urban Challenge. Specifially Caltech's entry to the challenge, Alice. During the pre-challenge testing campaign, Alice experienced a few situation, where it's behavior was unexpected and violated system-level guarantees.

<img src="alice_intersection.png" width="200"/>

The scenario we are analyzing in this notebook is Alice (red) approaching an intersection, where three cars are already waiting to enter the intersection, and Alice will need to give priority to these cars that are already present. Alice detects the cars and comes to a stop. BUT surprisingly, it immediately accelerates again and enters the intersection, violating its safety specification by entering the intersection when it does not have the right of way.

After analyzing this scenario, the Caltech team identified the cause for the failure. When Alice decelerated, its LIDARs tilted downwards and did not see the waiting cars anymore. Once Alice came to a stop, the LIDARs pointed straight ahead again and detected the cars. Alice was now under the assumption that it was the first to arrive at the intersection, and thus had the right of way, and entered the intersection.

We are setting up this example using Pacti and the diagnostics framework. First we will define the contracts for Alice's subsystems, compose them into Alice's system and then compose the entire system for multiple timesteps. We are given a violating system trace, containing the observations and system-level variables, and then diagnose the failure.

In [17]:
from pacti.contracts import PropositionalIoContract
from pacti.terms.propositions.propositions import PropositionalTerm
from pacti.iocontract import Var
import copy
import networkx as nx
from alice_example.alice_helperfunctions import build_composition_graph, connect_graphs, check_guarantee
from alice_example.system_trace import get_system_trace, get_internal_system_trace
from pacti.terms.propositions.propositions import _expr_to_str

First, let's set up the component contracts in Pacti. Alice's system consists of three components: Perception, the Planner, and the Tracker. Each of these components has assumptions on its inout variables, and provides guarantees tht can include its input and output variables. For more information about assume guarantee contracts and Pacti, please refer to www.pacti.org.

Alice's system and its interface variables are depicted below for a single timestep.

<img src="alice_single_timestep.png" width="600"/>

We will be composing Alice's system for multiple time steps, as shown in the figure below (including additional internal variables and system-level output variables shown in gray):

<img src="alice_multiple_timesteps.png" width="900"/>


### Perception

Let's start with the perception component, it is defined as:
$$\mathcal{C} = \{\{c_{T_1}^i, c_{T_2}^i, c_{T_3}^i, \text{poor\_visibility} \},\{c_{P_1}^i, c_{P_2}^i, c_{P_3}^i, x^i_1,\ldots, x^i_{100} \},\mathfrak{a}_{\text{Planner}}, \mathfrak{g}_{\text{Planner}} \},$$

where $\mathfrak{a}_{\text{Planner}} = \{  \neg \text{poor\_visibility} \} $.
The perception component guarantees are:
$$\mathfrak{g}_{\text{Planner}}= \{ c_{P_1}^i \Leftrightarrow c_{T_1}^i,\: c_{P_2}^i \Leftrightarrow c_{T_2}^i,\: c_{P_3}^i \Leftrightarrow c_{T_3}^i\} \cup \mathfrak{g}_{x},$$ 
where $\mathfrak{g}_{x} = \{x_k^i \vert 1 \leq k \leq 100 \}$.

The perception component's task is to make sure that the true cars ($c_T$) are detected correctly ($c_P$). This contract is defined below using Pacti:

In [18]:
def get_perception_contract(timestep):
    extra_perception_out_vars = ['x'+str(i) for i in range(100)]
    extra_guarantees = [f'{var}' for var in extra_perception_out_vars]
    perception = PropositionalIoContract.from_strings(
        input_vars=[f'car_l_T_t{timestep}', f'car_r_T_t{timestep}', f'car_s_T_t{timestep}', 'poor_visibility'],
        output_vars=[f'car_l_P_t{timestep}', f'car_r_P_t{timestep}', f'car_s_P_t{timestep}']+extra_perception_out_vars,
        assumptions=['~ poor_visibility'],
        guarantees=[f'car_l_T_t{timestep} <=> car_l_P_t{timestep}', f'car_s_T_t{timestep} <=> car_s_P_t{timestep}', f'car_r_T_t{timestep} <=> car_r_P_t{timestep}']+extra_guarantees,)
    return perception

### Planner

Next we will focus on the Planner component. The task of the planner is to track Alice's position in the priority queue and update it according to the observed other cars in the intersection.

The planner's output variables are $q^i = [q^i_1, q^i_2,q^i_3,q^i_4]$, indicator variables that correspond to Alice's position in the priority queue. If $q_1$ is true, Alice is in the first position and should take its turn. Otherwise Alice has to wait for its position to update until it reaches $q_1$. It also outputs the additional variables $y^i_1,\ldots, y^i_{100}$.

The input variables are $c_P^i$, the current observation of the other cars as provided by the planner, $c_P^{i-1}$ the observation from the last timestep, $q^{i-1}$ the last timestep's queue indicator variables, and $x^i_1,\ldots, x^i_{100}$, the additional variables from the perception component. 

The contract is defined as $\mathcal{C}_{\text{Planner}} = \{I_{\text{Planner}},O_{\text{Planner}},\mathfrak{a}_{\text{Planner}}, \mathfrak{g}_{\text{Planner}} \}$, where the planer assumptions are $$\mathfrak{a}_{\text{planner}} = \{(q^{i-1}_1 \land \neg q^{i-1}_2 \land \neg q^{i-1}_3 \land \neg q^{i-1}_4) \lor  (\neg q^{i-1}_1 \land q^{i-1}_2 \land \neg q^{i-1}_3 \land \neg q^{i-1}_4) \lor (\neg q^{i-1}_1 \land \neg q^{i-1}_2 \land q^{i-1}_3 \land \neg q^{i-1}_4) \lor (\neg q^{i-1}_1 \land \neg q^{i-1}_2 \land \neg q^{i-1}_3 \land q^{i-1}_4)\}.$$

The guarantees consist of the different observations that Alice could make, and how the queue inicator is updated. Specifically, if no car leaves the intersection, then Alice's position will remain the same:

$$
\begin{split}
\mathfrak{g}_0 = \{ &( c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_4^{i-1} \Rightarrow q_4^i, \\
&( c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_3^{i-1} \Rightarrow q_3^i, \\
&( c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_2^{i-1} \Rightarrow q_2^i, \\
&( c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_1^{i-1} \Rightarrow q_1^i \}
\end{split}
$$

In the case of one car leaving the intersection, Alice will move up one step in the queue:
\begin{split}
\mathfrak{g}_1 =  \{ & ( c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land ( \lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_4^{i-1}\Rightarrow q_3,\\
& ( c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land (  \lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_3^{i-1}\Rightarrow q_2,\\
& ( c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land ( \lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_2^{i-1}\Rightarrow q_1, \\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land ( \lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_4^{i-1}\Rightarrow q_3,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land ( \lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_3^{i-1}\Rightarrow q_2,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land ( \lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_2^{i-1}\Rightarrow q_1, \\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_4^{i-1}\Rightarrow q_3,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_3^{i-1}\Rightarrow q_2,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_2^{i-1}\Rightarrow q_1 \}.
\end{split} 


For two cars leaving the intersection, Alice will move up two steps in the queue:
\begin{split}
\mathfrak{g}_2 = & \{(c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land (\lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land (\lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_4^{i-1}\Rightarrow q_3,\\
& (c_{P_1}^i \Leftrightarrow c_{P_1}^{i-1}) \land (\lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land (\lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_3^{i-1}\Rightarrow q_1,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land (\lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_4^{i-1}\Rightarrow q_3,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( c_{P_2}^i \Leftrightarrow c_{P_2}^{i-1}) \land (\lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_3^{i-1}\Rightarrow q_1,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_4^{i-1}\Rightarrow q_3,\\
& (\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land ( \lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land ( c_{P_3}^i \Leftrightarrow c_{P_3}^{i-1}) \land q_3^{i-1}\Rightarrow q_1\}.
\end{split}

If all three cars leave the intersection, Alice will move from 3rd position into the first position:
\begin{split}
\mathfrak{g}_3 = \{((\lnot c_{P_1}^i \land c_{P_1}^{i-1}) \land (\lnot c_{P_2}^i \land c_{P_2}^{i-1}) \land (\lnot c_{P_3}^i \land c_{P_3}^{i-1}) \land q_4^{i-1}\Rightarrow q_1) \}.
\end{split}

Additionally, if Alice is in the first spot, it will stay in this spot until it takes its turn, 
$\mathfrak{g}_4 = \{q_1^{i-1} \Rightarrow q_1\}$

We are adding additional guarantees to the planner:
$$\mathfrak{g}_{extra} = \{x_k = y_k^i \vert 1 \leq k \leq 100 \}$$

Thus, $\mathfrak{g}_{\text{Planner}} = \mathfrak{g}_{0} \cup \mathfrak{g}_{1} \cup \mathfrak{g}_{2} \cup \mathfrak{g}_{3} \cup \mathfrak{g}_{4} \cup \mathfrak{g}_{extra}$.

In [19]:
def guarantee_generator(clist, timestep):
    """
    Generate a list of guarantees for Alice's planning component.
    """
    guarantees = []

    # for no changing cars, q has to stay the same
    status = ''
    for cx in clist:
        status += f'({cx}_t{timestep} <=> {cx}_t{timestep-1}) & '
    status = status[:-2]  # remove last '&'
    for q in [1,2,3,4]:
        guarantees.append(f'{status} & q_{q}_t{timestep-1} => q_{q}_t{timestep}')
    
    # guarantee generator for one changing car
    for c in clist:
        status = f'(~{c}_t{timestep} & {c}_t{timestep-1})'
        for cx in clist:
            if cx != c:
                status += f' & ({cx}_t{timestep} <=> {cx}_t{timestep-1})'
        for q in [2,3,4]:
            guarantees.append(f'{status} & q_{q}_t{timestep-1} => q_{q-1}_t{timestep}')

    # guarantee generator for two changing cars
    for c in clist:
        status = f'({c}_t{timestep} <=> {c}_t{timestep-1})'
        for cx in clist:
            if cx != c:
                status += f' & (~{cx}_t{timestep} & {cx}_t{timestep-1})'
        for q in [3,4]:
            guarantees.append(f'{status} & q_{q}_t{timestep-1} => q_{q-2}_t{timestep}')

    # guarantee generator for three changing cars
    status = ''
    for c in clist:
        status += f'(~{c}_t{timestep} & {c}_t{timestep-1}) &'
    status = status[:-2]  # remove last '&'
    guarantees.append(f'{status} & q_4_t{timestep-1} => q_1_t{timestep}')
    guarantees.append(f'(q_1_t{timestep} & ~q_2_t{timestep} & ~q_3_t{timestep} & ~q_4_t{timestep}) | (~q_1_t{timestep} & q_2_t{timestep} & ~q_3_t{timestep} & ~q_4_t{timestep}) | (~q_1_t{timestep} & ~q_2_t{timestep} & q_3_t{timestep} & ~q_4_t{timestep}) | (~q_1_t{timestep} & ~q_2_t{timestep} & ~q_3_t{timestep} & q_4_t{timestep})')

    # extra guarantees (that don/t have to do with couting cars)
    for i in range(100):
        guarantees.append(f'x{i} <=> y{i}')

    return guarantees

In [20]:
def get_planner_contract(timestep):
    extra_planner_in_vars = ['x'+str(i) for i in range(100)]
    extra_planner_out_vars = ['y'+str(i) for i in range(100)]
    planner = PropositionalIoContract.from_strings(
        input_vars=[f'car_l_P_t{timestep}', f'car_r_P_t{timestep}', f'car_s_P_t{timestep}', f'car_l_P_t{timestep-1}', f'car_r_P_t{timestep-1}', f'car_s_P_t{timestep-1}', f'q_1_t{timestep-1}', f'q_2_t{timestep-1}', f'q_3_t{timestep-1}', f'q_4_t{timestep-1}']+extra_planner_in_vars,
        output_vars=[f'q_1_t{timestep}', f'q_2_t{timestep}', f'q_3_t{timestep}', f'q_4_t{timestep}']+extra_planner_out_vars,
        assumptions=[f'(q_1_t{timestep-1} & ~q_2_t{timestep-1} & ~q_3_t{timestep-1} & ~q_4_t{timestep-1}) | (~q_1_t{timestep-1} & q_2_t{timestep-1} & ~q_3_t{timestep-1} & ~q_4_t{timestep-1}) | (~q_1_t{timestep-1} & ~q_2_t{timestep-1} & q_3_t{timestep-1} & ~q_4_t{timestep-1}) | (~q_1_t{timestep-1} & ~q_2_t{timestep-1} & ~q_3_t{timestep-1} & q_4_t{timestep-1})'],
        guarantees=guarantee_generator(['car_l_P', 'car_r_P', 'car_s_P'], timestep))
    return planner

### Tracker

The tracker component is responsible for setting Alice's speed. The input variables are Alice's spot in the queue $q_1^i$, and $y^i_1,\ldots, y^i_{100}$, the planner's additional output variables.
It's output variable is $v^i$, where $v^i=1$ corresponds to a positive speed, and $v^i=0$ corresponds to Alice staying in place, and $z^i_1,\ldots, z^i_{100}$, the additional tracker output.
The contract is defined as $\mathcal{C}_{\text{Tracker}} = \{i,o,\mathfrak{a}_{\text{Tracker}}, \mathfrak{g}_{\text{Tracker}} \}$, where $\mathfrak{g}_{\text{Tracker}} = \{ q_1^i \Leftrightarrow v^i \} \cup \{z_k^i \vert 1 \leq k \leq 100 \}$.

In [21]:
def get_tracker_contract(timestep):
    extra_tracker_in_vars = ['y'+str(i) for i in range(100)]
    extra_tracker_out_vars = ['z'+str(i)+'_'+str(timestep) for i in range(100)]
    extra_guarantees = [f'y{i} <=> {var} ' for i,var in enumerate(extra_tracker_out_vars)]
    tracker = PropositionalIoContract.from_strings(
        input_vars=[f'q_1_t{timestep}', f'q_2_t{timestep}',f'q_3_t{timestep}', f'q_4_t{timestep}', 'icy_roads']+extra_tracker_in_vars,
        output_vars=[f'v_t{timestep}']+extra_tracker_out_vars,
        assumptions=['~icy_roads'],
        guarantees=[f'q_1_t{timestep} <=> v_t{timestep}']+extra_guarantees)
    return tracker

Now we can compose these component contracts for a single timestep. First we compose the Perception and the Planner, and then the composed contract with the Tracker. For the intermediate time steps, we need to keep the internal $c_P^i$ and $q^i$ variables, as they are the input to the next time step. For the final timestep, the outputs are only the obersevations $v^i$ and $z^i$.

In [22]:
def get_system_sequenced(timestep):
    """
    Get the system contract and diagnostics graph for one timestep, keeping internal variables.
    """
    perception = get_perception_contract(timestep)
    planner = get_planner_contract(timestep)

    perception_and_planner, G1 = perception.compose_diagnostics(planner, vars_to_keep=[f'q_1_t{timestep}', f'q_2_t{timestep}', f'q_3_t{timestep}', f'q_4_t{timestep}',f'car_l_P_t{timestep}', f'car_r_P_t{timestep}', f'car_s_P_t{timestep}'])
    for i in G1.nodes():
        if i not in perception_and_planner.g.terms and i not in perception_and_planner.a.terms:
            G1.nodes[i]['output'] = False
    # build the composition graph for the first composition
    contractdict = {'self': f'perception_{timestep}', 'other': f'planner_{timestep}', 'composition': f'perception_and_planner_{timestep}'}
    G1_c = build_composition_graph(G1, 'a', contractdict, system_level=False)

    tracker = get_tracker_contract(timestep)
    perception_planner_and_tracker, G2 = perception_and_planner.compose_diagnostics(tracker, vars_to_keep = [f'q_1_t{timestep}', f'q_2_t{timestep}', f'q_3_t{timestep}', f'q_4_t{timestep}', f'car_l_P_t{timestep}', f'car_r_P_t{timestep}', f'car_s_P_t{timestep}'])
    for i in G2.nodes():
        if i not in perception_planner_and_tracker.g.terms and i not in perception_planner_and_tracker.a.terms:
            G2.nodes[i]['output'] = False
    # build the composition graph for the second composition
    contractdict = {'self': f'perception_and_planner_{timestep}', 'other': f'tracker_{timestep}', 'composition': f'system_{timestep}'}
    G2_c = build_composition_graph(G2, 'b', contractdict, system_level=False)

    return perception_planner_and_tracker, perception_and_planner, G2_c, G1_c

In [23]:
def get_system_final_timestep(timestep):
    """
    Get the system contract for the last timestep (do not keep the internal q and car_P variables).
    """
    perception = get_perception_contract(timestep)
    planner = get_planner_contract(timestep)

    perception_and_planner, G1 = perception.compose_diagnostics(planner)#, vars_to_keep=[f'q_1_t{timestep}', f'q_2_t{timestep}', f'q_3_t{timestep}', f'q_4_t{timestep}',f'car_l_P_t{timestep}', f'car_r_P_t{timestep}', f'car_s_P_t{timestep}'])
    for i in G1.nodes():
        if i not in perception_and_planner.g.terms and i not in perception_and_planner.a.terms:
            G1.nodes[i]['output'] = False
    # build the composition graph for the first composition
    contractdict = {'self': f'perception_{timestep}', 'other': f'planner_{timestep}', 'composition': f'perception_and_planner_{timestep}'}
    G1_c = build_composition_graph(G1, 'c', contractdict, system_level=False)

    tracker = get_tracker_contract(timestep)
    perception_planner_and_tracker, G2 = perception_and_planner.compose_diagnostics(tracker)
    for i in G2.nodes(): # filter out every guarantee that is dropped
        if i not in perception_planner_and_tracker.g.terms and i not in perception_planner_and_tracker.a.terms:
            G2.nodes[i]['output'] = False
    # build the composition graph for the second composition
    contractdict = {'self': f'perception_and_planner_{timestep}', 'other': f'tracker_{timestep}', 'composition': f'system_{timestep}'}
    G2_c = build_composition_graph(G2, 'd', contractdict, system_level=False)    

    return perception_planner_and_tracker, perception_and_planner, G2_c, G1_c

Now we can use the functions above to compose the system first per time step, and then compose the system contract for multiple time steps and construct the diagnostics graph.

In [24]:
timestep = 1

# compose first timestep
sys, perception_and_planner, G2, G1 = get_system_sequenced(timestep)
G_sys_1 = connect_graphs(G2, G1)
# compose second timestep
sys2, perception_and_planner2, G2_2, G1_2 = get_system_final_timestep(timestep+1)
G_sys_2 = connect_graphs(G2_2, G1_2)

# compose two timestep contracts
full_sys, G_sys = sys.compose_diagnostics(sys2)
contractdict = {'self': f'system_{timestep}', 'other': f'system_{timestep+1}', 'composition': 'full_system'}
G_sys = build_composition_graph(G_sys, 'f', contractdict, system_level=True)
# get the diagnostics graph G
G = connect_graphs(G_sys, G_sys_1, G_sys_2)

We can now use the diagnostic graph G to map the component-level terms and the system-level terms to their corresponding nodes:

In [25]:
sys_level_guarantee_terms = [node for node in G.nodes() if G.nodes[node]['system_level']=='True' and G.nodes[node]['type']=='guarantee' and G.nodes[node]['output']=='True']
component_level_input_terms = [node for node in G.nodes() if G.nodes[node]['input']=='True' and G.nodes[node]['contract'] in ['perception_1', 'planner_1', 'tracker_1', 'perception_2', 'planner_2', 'tracker_2']]


## Evaluating the system-level guarantees

We are given the following system-level trace, where 


| Time step $i$ | poor_visibility | icy_roads | $c_{T_1}^i$ | $c_{T_2}^i$ | $c_{T_3}^i$ | $c_{P_1}^i$ | $c_{P_2}^i$ | $c_{P_3}^i$ | $q_1^i$ | $q_2^i$ | $q_3^i$ | $q_4^i$ | $v^i$ | $z^1_{0} \ldots z^1_{100}$ | $z^2_{0} \ldots z^2_{100}$ |
|---------------|------------------|----------|-------------|-------------|-------------|-------------|-------------|-------------|---------|---------|---------|---------|-------|------|------|
| 0             | 0                | 0        | 1           | 1           | 1           | 1           | 1           | 1           | 0       | 0       | 0       | 1       | 0     | $1 \ldots 1$    | $1 \ldots 1$    |
| 1             | 0                | 0        | 1           | 1           | 1           | --          | --          | --          | --      | --      | --      | --      | 0     | $1 \ldots 1$    | $1 \ldots 1$    |
| 2             | 0                | 0        | 1           | 1           | 1           | --          | --          | --          | --      | --      | --      | --      | 1     | $1 \ldots 1$    | $1 \ldots 1$    |

Note that we do not have access to the internal variables at the system level (the -- cells in the table).
Now that everything is set up, let's load the system trace and check the observed behavior! We will load this trace and analyze the guarantees.

In [26]:
trace = get_system_trace()
i = 0
behavior = {Var(key) : trace[key][i] for key in trace.keys()}
print(behavior)

{<Var poor_visibility>: 0, <Var icy_roads>: 0, <Var car_l_P_t0>: 1, <Var car_r_P_t0>: 1, <Var car_s_P_t0>: 1, <Var car_l_T_t1>: 1, <Var car_r_T_t1>: 1, <Var car_s_T_t1>: 1, <Var car_l_T_t2>: 1, <Var car_r_T_t2>: 1, <Var car_s_T_t2>: 1, <Var q_1_t0>: 0, <Var q_2_t0>: 0, <Var q_3_t0>: 0, <Var q_4_t0>: 1, <Var v_t1>: 0, <Var v_t2>: 1, <Var z0_1>: 1, <Var z0_2>: 1, <Var z1_1>: 1, <Var z1_2>: 1, <Var z2_1>: 1, <Var z2_2>: 1, <Var z3_1>: 1, <Var z3_2>: 1, <Var z4_1>: 1, <Var z4_2>: 1, <Var z5_1>: 1, <Var z5_2>: 1, <Var z6_1>: 1, <Var z6_2>: 1, <Var z7_1>: 1, <Var z7_2>: 1, <Var z8_1>: 1, <Var z8_2>: 1, <Var z9_1>: 1, <Var z9_2>: 1, <Var z10_1>: 1, <Var z10_2>: 1, <Var z11_1>: 1, <Var z11_2>: 1, <Var z12_1>: 1, <Var z12_2>: 1, <Var z13_1>: 1, <Var z13_2>: 1, <Var z14_1>: 1, <Var z14_2>: 1, <Var z15_1>: 1, <Var z15_2>: 1, <Var z16_1>: 1, <Var z16_2>: 1, <Var z17_1>: 1, <Var z17_2>: 1, <Var z18_1>: 1, <Var z18_2>: 1, <Var z19_1>: 1, <Var z19_2>: 1, <Var z20_1>: 1, <Var z20_2>: 1, <Var z21_1>: 1

First, let's check the system-level assumptions and system-level guarantees:

In [27]:
if full_sys.a.contains_behavior(behavior):
    print('Assumptions are satisfied')
else:
    print('System-level assumptions are NOT satisfied')
if full_sys.g.contains_behavior(behavior):
    print('System-level guarantees are satisfied')
else:
    print('System-level guarantees are NOT satisfied')

Assumptions are satisfied
System-level guarantees are NOT satisfied


We see that the assumptions are satisfied, but the guarantees are not! Now, let's check the individual system-level guarantees to find which one we need to debug:

In [28]:
violated_gs = []
for i,term in enumerate(full_sys.g.terms):
    if not check_guarantee(term, behavior):
        nodes = [node for node in sys_level_guarantee_terms if G.nodes[node]['term']==_expr_to_str(term.expression)]
        violated_node = nodes[0]
        violated_gs.append(violated_node)
        print(f'Guarantee No. {i+1}/{len(full_sys.g.terms)}: is violated (node {violated_node})')
    else:
        print(f'Guarantee No. {i+1}/{len(full_sys.g.terms)} is satisfied')

print(f'Need to diagnose: Node {violated_gs}')

Guarantee No. 1/212 is satisfied
Guarantee No. 2/212 is satisfied
Guarantee No. 3/212 is satisfied
Guarantee No. 4/212 is satisfied
Guarantee No. 5/212 is satisfied
Guarantee No. 6/212 is satisfied
Guarantee No. 7/212 is satisfied
Guarantee No. 8/212 is satisfied
Guarantee No. 9/212 is satisfied
Guarantee No. 10/212 is satisfied
Guarantee No. 11/212 is satisfied
Guarantee No. 12/212 is satisfied
Guarantee No. 13/212 is satisfied
Guarantee No. 14/212 is satisfied
Guarantee No. 15/212 is satisfied
Guarantee No. 16/212 is satisfied
Guarantee No. 17/212 is satisfied
Guarantee No. 18/212 is satisfied
Guarantee No. 19/212 is satisfied
Guarantee No. 20/212 is satisfied
Guarantee No. 21/212 is satisfied
Guarantee No. 22/212 is satisfied
Guarantee No. 23/212 is satisfied
Guarantee No. 24/212 is satisfied
Guarantee No. 25/212 is satisfied
Guarantee No. 26/212 is satisfied
Guarantee No. 27/212 is satisfied
Guarantee No. 28/212 is satisfied
Guarantee No. 29/212 is satisfied
Guarantee No. 30/212 is

We have identified the violated system-level guarantee, to show how complex these terms can get, let's print the term:

In [29]:
for node in violated_gs:
    print(f'Diagnosing: {G.nodes[node]["term"]}')

Diagnosing: ((car_l_T_t1) & (car_s_T_t1) & (car_r_T_t1) & (~ v_t1) & (((~ car_l_T_t2) & (~ car_s_T_t2) & (~ car_r_T_t2) & (v_t2) & (((~ car_l_T_t2) & (~ car_s_T_t2) & (~ car_r_T_t2)) | ((car_l_T_t2) & (~ car_s_T_t2) & (~ car_r_T_t2)) | ((~ car_l_T_t2) & (car_s_T_t2) & (~ car_r_T_t2)) | ((car_l_T_t2) & (car_s_T_t2) & (~ car_r_T_t2)) | ((~ car_l_T_t2) & (~ car_s_T_t2) & (car_r_T_t2)) | ((car_l_T_t2) & (~ car_s_T_t2) & (car_r_T_t2)) | ((~ car_l_T_t2) & (car_s_T_t2) & (car_r_T_t2)) | ((car_l_T_t2) & (car_s_T_t2) & (car_r_T_t2))) & (((~ car_l_T_t2) & (~ car_s_T_t2) & (~ car_r_T_t2)) | ((car_l_T_t2) & (~ car_s_T_t2) & (~ car_r_T_t2)) | ((~ car_l_T_t2) & (car_s_T_t2) & (~ car_r_T_t2)) | ((car_l_T_t2) & (car_s_T_t2) & (~ car_r_T_t2)) | ((~ car_l_T_t2) & (~ car_s_T_t2) & (car_r_T_t2)) | ((car_l_T_t2) & (~ car_s_T_t2) & (car_r_T_t2)) | ((~ car_l_T_t2) & (car_s_T_t2) & (car_r_T_t2)) | ((car_l_T_t2) & (car_s_T_t2) & (car_r_T_t2))) & (((~ car_l_T_t2) & (~ car_s_T_t2) & (~ car_r_T_t2)) | ((car_l_T_t

We can now use the diagnostics approach to identify which component-level terms we need to check.

In [30]:
to_check = []
for sink in violated_gs:
    print(f'Checking for system_level violated guarantee: Node {sink}')
    for src in component_level_input_terms:
        if nx.has_path(G, src, sink):
            print(f'Found relevant term: Need to check {G.nodes[src]["term"]} from {G.nodes[src]["contract"]}')
            if src not in to_check:
                to_check.append(src)

Checking for system_level violated guarantee: Node f275o
Found relevant term: Need to check (car_l_T_t1) <=> (car_l_P_t1) from perception_1
Found relevant term: Need to check (car_s_T_t1) <=> (car_s_P_t1) from perception_1
Found relevant term: Need to check (car_r_T_t1) <=> (car_r_P_t1) from perception_1
Found relevant term: Need to check ((q_1_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_1_t1) from planner_1
Found relevant term: Need to check ((q_2_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_2_t1) from planner_1
Found relevant term: Need to check ((q_3_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_3_t1) from planner_1
Found relevant term: Need to check ((q_4_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_4_t1) from plan

In [31]:
print(f'We need to evaluate {len(set(to_check))/len(component_level_input_terms)*100} % component level terms, {len(to_check)} out of total {len(component_level_input_terms)}')

We need to evaluate 7.621951219512195 % component level terms, 50 out of total 656


Now let's check the 50 relevant terms to find the cause for the failure:

In [32]:
internal_trace = get_internal_system_trace()
trace = trace | internal_trace
behavior = {Var(key) : trace[key][0] for key in trace.keys()}


for node in to_check:
    term = PropositionalTerm(G.nodes[node]['term'])
    if not check_guarantee(term, behavior):
        print(f'*** Violated Guarantee {G.nodes[node]["term"]} from {G.nodes[node]["contract"]}')
    else:
        print(f'--- Satisfied Guarantee {G.nodes[node]["term"]} from {G.nodes[node]["contract"]}')

*** Violated Guarantee (car_l_T_t1) <=> (car_l_P_t1) from perception_1
*** Violated Guarantee (car_s_T_t1) <=> (car_s_P_t1) from perception_1
*** Violated Guarantee (car_r_T_t1) <=> (car_r_P_t1) from perception_1
--- Satisfied Guarantee ((q_1_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_1_t1) from planner_1
--- Satisfied Guarantee ((q_2_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_2_t1) from planner_1
--- Satisfied Guarantee ((q_3_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_3_t1) from planner_1
--- Satisfied Guarantee ((q_4_t0) & ((car_l_P_t1) <=> (car_l_P_t0)) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0))) => (q_4_t1) from planner_1
--- Satisfied Guarantee ((~ car_l_P_t1) & (q_2_t0) & (car_l_P_t0) & ((car_r_P_t1) <=> (car_r_P_t0)) & ((car_s_P_t1) <=> (car_s_P_t0

Which leads us to identify the perception system in the first timestep as the culprit for the problem!